## **Import Library**

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
!pip install rank-bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import re
import pandas as pd
import numpy as np

In [ ]:
from rank_bm25 import BM25Okapi

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer()

In [ ]:
datasets = pd.read_csv('/content/bengkel_new.xlsx - Sheet1.csv')
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   nama             150 non-null    object 
 1   photo            150 non-null    object 
 2   alamat           150 non-null    object 
 3   latitude         150 non-null    float64
 4   longitude        150 non-null    float64
 5   kontak           147 non-null    float64
 6   jam operasional  143 non-null    object 
 7   fitur            150 non-null    object 
 8   label            150 non-null    object 
dtypes: float64(3), object(6)
memory usage: 10.7+ KB


In [ ]:
datasets.head(5)

,nama,photo,alamat,latitude,longitude,kontak,jam operasional,fitur,label
0,bengkel mobil mas mono,https://www.google.com/maps/place/Bengkel+Mobi...,"Jl. Bekasi Tim. Raya No.176, RT.1/RW.6, Cipina...",-6.214579,106.886882,8.211367e+10,"senin - sabtu 08.30 - 17.30, minggu tutup",menerima segala jenis perbaikan masalah mobil,int64
1,bengkel mobil pawang,https://www.google.com/maps/place/Bengkel+Mobi...,"Jl. Delima Raya No.10, RT.2/RW.5, Malaka Sari,...",-6.221298,106.926501,2.186005e+08,senin - minggu 08.00 - 17.00,menerima segala jenis perbaikan masalah mobil,int64
2,bengkel mobil anugerah 2,https://www.google.com/maps/place/Bengkel+Mobi...,"4, Jl. Klp. Sawit Raya No.21, RT.4/RW.7, Pd. K...",-6.240361,106.926209,8.138057e+10,"senin - sabtu 08.00 - 17.00, minggu tutup","ganti oli, tune up, overhaul, dan perbaikan tr...",int64
3,bengkel mobil almet spesialis matic,https://www.google.com/maps/place/Bengkel+Mobi...,"Jl. Jend. Basuki Rachmat No.2, RW.6, Cipinang ...",-6.227173,106.883561,8.129113e+10,senin - minggu 07.30 - 17.00,"servis perbaikan AC, perbaikan rem mobil, powe...",int64
4,bengkel mobil 32 motor,https://www.google.com/maps/place/Bengkel+Mobi...,"Jl. Cipinang Baru Raya No.32, RW.12, Cipinang,...",-6.204389,106.887831,2.147180e+08,"senin - sabtu 08.00 - 17.00, minggu tutup","body repair, ketok magic, pengecatan mobil",int64


In [ ]:
Sentence = datasets.nama[0]
Word = word_tokenize(Sentence)
print(Word)

['bengkel', 'mobil', 'mas', 'mono']


In [ ]:
def stopwords_split_chars_removal(lst):
  corpus=[]
  for i in range(len(datasets)):
    review=re.sub('[^a-zA-Z]',' ',datasets.nama[i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(i) for i in review if i not in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

  return corpus

In [ ]:
len(corpus)

150

In [ ]:
corpus = []
for bengkel in datasets['nama']:
  # Tokenization
  words = word_tokenize(bengkel)

  # Stopwords Split Chars Removal
  words = stopwords_split_chars_removal(bengkel)

  # Stemming
  final_lst = []
  for word in words:
      final_lst.append(ps.stem(word))
        
  corpus.append(' '.join(final_lst))

def news_search(query, corpus, n, news_data):
  tokenized_corpus = [doc.split(" ") for doc in corpus]
  bm25 = BM25Okapi(tokenized_corpus)
  tokenized_query = query.split(" ")
  doc_scores = bm25.get_scores(tokenized_query)
  ind = np.argpartition(doc_scores, -n)[-n:]
  return news_data['nama'][ind]

In [ ]:
Query = input("Enter your query for news search: ").lower()
n = 15
print(f'Displaying top {n} articles \n')
result = news_search(Query, corpus, n, datasets)
for ans in result:
  print(ans)
  print()

Enter your query for news search: mobil
Displaying top 15 articles 

pt. asido nayochi

bung mekanik official

bengkel mobil siliwangi auto

R-Speed PRO Garage

bengkel pospera

bengkel cat mobil 123

bengkel mobil haya automotor (mekanik mobil panggilan)

Bengkel Mobil BOS Kavling DKI

Dua Rajo Auto CARe

bengkel spesialis mobil matic

Bengkel Mobil BOS Kedoya

Bengkel Mobil Mutiara Jaya

KJS Motor - Spesialis Bengkel Mesin & Transmisi Mobil Jakarta

bengkel mobil rahayu

bengkel mobil sinar bahagia

